In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from icesea2.functions import volume_to_sle, calc_grid_area_geo
from icesea2.gridtools import region_mask

# Analytics

In [9]:
x, y = batchelor_topo_0_ds.x.values, batchelor_topo_0_ds.y.values
dA = (abs(x[1] - x[0]) * abs(y[1] - y[0]))
shape = batchelor_topo_0_ds.ice_thickness[0,0].shape
# barants-kara sea
mask_bks = np.zeros(shape)
mask_bks[588:, :] = 1

# fennoscandia
mask_f = np.zeros(shape)
mask_f[:588, 294:] = 1

# british-irish
mask_bi = np.zeros(shape)
mask_bi[:588, :294] = 1

In [30]:
def output_analytics(ds, output):
    n_ensemble = ds.ice_thickness.shape[0]
    time = ds.time.values

    x, y = ds.x.values, ds.y.values
    dx = np.abs(x[1] - x[0])
    dy = np.abs(y[1] - y[0])
    dA = dx * dy

    ice_thickness_mean = ds.ice_thickness.mean(axis=0).values
    ice_thickness_std = ds.ice_thickness.std(axis=0).values
    shearstress_mean = ds.basal_shearstress.mean(axis=0).values
    shearstress_std = ds.basal_shearstress.std(axis=0).values

    ice_volume = ds.ice_thickness.sum(axis=(2,3)).values * dA
    ice_volume_sle = volume_to_sle(ice_volume)
    ice_volume_mean = ice_volume.mean(axis=0)
    ice_volume_sle_mean = ice_volume_sle.mean(axis=0)

    region_ice_volume = np.zeros((3, *ds.ice_thickness.shape[:2]))
    region_ice_volume[0] = (dA * ds.ice_thickness * mask_bks).sum(axis=(2,3)).values
    region_ice_volume[1] = (dA * ds.ice_thickness * mask_f).sum(axis=(2,3)).values
    region_ice_volume[2] = (dA * ds.ice_thickness * mask_bi).sum(axis=(2,3)).values

    region_ice_volume_sle = volume_to_sle(region_ice_volume)

    analytics_ds = xr.Dataset(
        data_vars=dict(
            ice_thickness_mean=(
                ["time", "y", "x"],
                ice_thickness_mean,
            ),
            basal_shearstress_mean=(
                ["time", "y", "x"],
                shearstress_mean,
            ),
            ice_thickness_std=(
                ["time", "y", "x"],
                ice_thickness_std,
            ),
            basal_shearstress_std=(
                ["time", "y", "x"],
                shearstress_std,
            ),
            ice_volume=(
                ["ensemble_index", "time"],
                ice_volume,
            ),
            ice_volume_sle=(
                ["ensemble_index", "time"],
                ice_volume_sle,
            ),
            region_ice_volume=(
                ["region", "ensemble_index", "time"],
                region_ice_volume
            ),
            region_ice_volume_sle=(
                ["region", "ensemble_index", "time"],
                region_ice_volume_sle
            )
        ),
        coords={
            "ensemble_index": list(range(n_ensemble)),
            "time": time,
            "region": ['bks', 'f', 'bi'],
            "y": y,
            "x": x,
        },
        attrs=None,
    )
    analytics_ds.rio.write_crs(ds.rio.crs, inplace=True)
    analytics_ds.to_netcdf(output)

In [43]:
from pathlib import Path
datasets = [
    Path("../runs/batchelor/batchelor_topo_0.nc"),
    Path("../runs/batchelor/batchelor_topo_1.nc"),
    Path("../runs/glac1d/glac1d_icesheet.nc"),
    Path("../runs/ice6g/ice6g_icesheet.nc")
]
for dataset in datasets:
    ds = xr.open_dataset(dataset)
    output_analytics(ds, f"{dataset.stem}_analytics.nc")

In [7]:
batchelor_topo_1_ds = xr.open_dataset("batchelor_topo_1.nc")

ice_thickness_mean = batchelor_topo_1_ds.ice_thickness.mean(axis=0).values
topography_mean = batchelor_topo_1_ds.bedrock_topography.mean(axis=0).values
shearstress_mean = batchelor_topo_1_ds.basal_shearstress.mean(axis=0).values

ice_thickness_std = batchelor_topo_1_ds.ice_thickness.std(axis=0).values
topography_std = batchelor_topo_1_ds.bedrock_topography.std(axis=0).values
shearstress_std = batchelor_topo_1_ds.basal_shearstress.std(axis=0).values

ice_volume = batchelor_topo_1_ds.ice_thickness.sum(axis=(2,3)).values * dA
ice_volume_sle = volume_to_sle(ice_volume)
ice_volume_mean = ice_volume.mean(axis=0)
ice_volume_sle_mean = ice_volume_sle.mean(axis=0)

batchelor_topo_1_analytics_ds = xr.Dataset(
    data_vars=dict(
        ice_thickness_mean=(
            ["time", "y", "x"],
            ice_thickness_mean,
        ),
        bedrock_topography_mean=(
            ["time", "y", "x"],
            topography_mean,
        ),
        basal_shearstress_mean=(
            ["time", "y", "x"],
            shearstress_mean,
        ),
        ice_thickness_std=(
            ["time", "y", "x"],
            ice_thickness_std,
        ),
        bedrock_topography_std=(
            ["time", "y", "x"],
            topography_std,
        ),
        basal_shearstress_std=(
            ["time", "y", "x"],
            shearstress_std,
        ),
        ice_volume=(
            ["ensemble_index", "time"],
            ice_volume,
        ),
        ice_volume_mean=(
            ["time"],
            ice_volume_mean,
        ),
        ice_volume_sle=(
            ["ensemble_index", "time"],
            ice_volume_sle,
        ),
        ice_volume_sle_mean=(
            ["time"],
            ice_volume_sle_mean,
        ),
    ),
    coords={
        "ensemble_index": list(range(200)),
        "time": time,
        "y": y,
        "x": x,
    },
    attrs=None,
)
batchelor_topo_1_analytics_ds.rio.write_crs(ds.rio.crs, inplace=True)
batchelor_topo_1_analytics_ds.to_netcdf(f"batchelor_topo_1_analytics.nc")


/nfs/see-fs-02_users/cm15ogp/conda/envs/py3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


# LGM Analytics

In [32]:
ice6g_icesheet_ds = xr.open_dataset("paper1_data/icesheet_runs/ice6g/ice6g_icesheet.nc")
ice6g_ds = xr.open_dataset("paper1_data/ice_models/ice6g_laea_5km_eurasia.nc")

In [90]:
ice6g_icesheet_analytics_ds = xr.open_dataset("ice6g_icesheet_analytics.nc")

In [34]:
ice6g_diff = ice6g_icesheet_ds.ice_thickness - ice6g_ds.ice_thickness

In [111]:
ice6g_icesheet_ds.sel(time=-16000).mean(axis=0)

<xarray.Dataset>
Dimensions:             (ensemble_index: 200, x: 1086, y: 1216)
Coordinates:
  * ensemble_index      (ensemble_index) int64 0 1 2 3 4 ... 195 196 197 198 199
    time                int64 -16000
  * y                   (y) float64 -4.723e+06 -4.718e+06 ... 1.352e+06
  * x                   (x) float64 -1.265e+06 -1.26e+06 ... 4.155e+06 4.16e+06
    spatial_ref         int64 0
Data variables:
    ice_thickness       (ensemble_index, y, x) float64 0.0 0.0 0.0 ... 0.0 0.0
    bedrock_topography  (y, x) float64 ...
    basal_shearstress   (ensemble_index, y, x) float64 ...
Attributes:
    grid_mapping:  spatial_ref

In [135]:
ice6g_nonzero = np.count_nonzero(ice6g_ds.ice_thickness.sel(time=[-22000,-20000,-18000,-16000]).values, axis=(1,2))
glac1d_nonzero = np.count_nonzero(glac1d_ds.ice_thickness.sel(time=[-22000,-20000,-18000,-16000]).values, axis=(1,2))

ice6g_area = ice6g_nonzero * 5000**2
glac1d_area = glac1d_nonzero * 5000**2

ice6g_volume = volume_to_sle((ice6g_ds.ice_thickness.sel(time=[-22000,-20000,-18000,-16000]).values * 5000**2).sum(axis=(1,2)))
glac1d_volume = volume_to_sle((glac1d_ds.ice_thickness.sel(time=[-22000,-20000,-18000,-16000]).values * 5000**2).sum(axis=(1,2)))


ice6g_nonzero/glac1d_nonzero


array([0.56327848, 0.54837197, 0.57114028, 0.54120492])

In [66]:
shape = ice6g_diff.values[0,0].shape
mask_bks = np.zeros(shape)
mask_bks[588:, :] = 1

# fennoscandia
mask_f = np.zeros(shape)
mask_f[:588, 294:] = 1

# british-irish
mask_bi = np.zeros(shape)
mask_bi[:588, :294] = 1

In [67]:
ice6g_diff_bks = ice6g_diff * mask_bks
ice6g_diff_f = ice6g_diff * mask_f
ice6g_diff_bi = ice6g_diff * mask_bi

In [24]:
def lgm_icesheet_analytics(ds):
    time = ds.time.values
    x = ds.x.values
    y = ds.y.values

    dx = np.abs(x[1] - x[0])
    dy = np.abs(y[1] - y[0])
    dA = dx * dy

    regions = ['bks', 'f', 'bi']
    
    shape = ds.ice_thickness[0,0].shape
    # barants-kara sea
    mask_bks = np.zeros(shape)
    mask_bks[588:, :] = 1

    # fennoscandia
    mask_f = np.zeros(shape)
    mask_f[:588, 294:] = 1

    # british-irish
    mask_bi = np.zeros(shape)
    mask_bi[:588, :294] = 1
    
    ice_thickness_mean = ds.ice_thickness.mean(axis=0).values
    shearstress_mean = ds.basal_shearstress.mean(axis=0).values

    ice_thickness_std = ds.ice_thickness.std(axis=0).values
    shearstress_std = ds.basal_shearstress.std(axis=0).values

    ice_volume = ds.ice_thickness.sum(axis=(2,3)).values * dA
    ice_volume_sle = volume_to_sle(ice_volume)
    ice_volume_mean = ice_volume.mean(axis=0)
    ice_volume_sle_mean = ice_volume_sle.mean(axis=0)

    region_ice_volume = np.zeros((3,*ice_volume.shape))
    region_ice_volume[0] = (ds.ice_thickness * mask_bks).sum(axis=(2,3)).values * dA
    region_ice_volume[1] = (ds.ice_thickness * mask_f).sum(axis=(2,3)).values * dA
    region_ice_volume[2] = (ds.ice_thickness * mask_bi).sum(axis=(2,3)).values * dA

    region_ice_volume_sle = volume_to_sle(region_ice_volume)
    region_ice_volume_mean = region_ice_volume.mean(axis=1)
    region_ice_volume_mean_sle = region_ice_volume_sle.mean(axis=1)

    analytics_ds = xr.Dataset(
        data_vars=dict(
            ice_thickness_mean=(
                ["time", "y", "x"],
                ice_thickness_mean,
            ),
            basal_shearstress_mean=(
                ["time", "y", "x"],
                shearstress_mean,
            ),
            ice_thickness_std=(
                ["time", "y", "x"],
                ice_thickness_std,
            ),
            basal_shearstress_std=(
                ["time", "y", "x"],
                shearstress_std,
            ),
            ice_volume=(
                ["ensemble_index", "time"],
                ice_volume,
            ),
            ice_volume_mean=(
                ["time"],
                ice_volume_mean,
            ),
            ice_volume_sle=(
                ["ensemble_index", "time"],
                ice_volume_sle,
            ),
            ice_volume_sle_mean=(
                ["time"],
                ice_volume_sle_mean,
            ),
            region_ice_volume=(
            ["region", "ensemble_index", "time"],
                region_ice_volume,
            ),
        ),
        coords={
            "ensemble_index": list(range(len(ds.ensemble_index))),
            "time": time,
            "y": y,
            "x": x,
            "region": regions
        },
        attrs=None,
    )
    analytics_ds.rio.write_crs(ds.rio.crs, inplace=True)
    return analytics_ds

In [11]:
def lgm_model_analytics(ds, time):
    x = ds.x.values
    y = ds.y.values

    dx = np.abs(x[1] - x[0])
    dy = np.abs(y[1] - y[0])
    dA = dx * dy

    regions = ['bks', 'f', 'bi']
    
    shape = ds.ice_thickness[0].shape
    # barants-kara sea
    mask_bks = np.zeros(shape)
    mask_bks[588:, :] = 1

    # fennoscandia
    mask_f = np.zeros(shape)
    mask_f[:588, 294:] = 1

    # british-irish
    mask_bi = np.zeros(shape)
    mask_bi[:588, :294] = 1

    ice_volume = ds.ice_thickness.sel(time=time).sum(axis=(1,2)).values * dA
    ice_volume_sle = volume_to_sle(ice_volume)

    region_ice_volume = np.zeros((3,*ice_volume.shape))
    region_ice_volume[0] = (ds.ice_thickness.sel(time=time) * mask_bks).sum(axis=(1,2)).values * dA
    region_ice_volume[1] = (ds.ice_thickness.sel(time=time) * mask_f).sum(axis=(1,2)).values * dA
    region_ice_volume[2] = (ds.ice_thickness.sel(time=time) * mask_bi).sum(axis=(1,2)).values * dA

    region_ice_volume_sle = volume_to_sle(region_ice_volume)

    analytics_ds = xr.Dataset(
        data_vars=dict(
            ice_volume=(
                ["time"],
                ice_volume,
            ),
            ice_volume_sle=(
                ["time"],
                ice_volume_sle,
            ),
            region_ice_volume=(
            ["region", "time"],
                region_ice_volume,
            ),
            region_ice_volume_sle=(
            ["region", "time"],
                region_ice_volume_sle,
            ),
        ),
        coords={
            "time": time,
            "region": regions
        },
        attrs=None,
    )
    return analytics_ds

In [ ]:
ice6g_icesheet_ds = xr.open_dataset("paper1_data/icesheet_runs/ice6g/ice6g_icesheet.nc")
ice6g_icesheet_analytics_ds = lgm_icesheet_analytics(ice6g_icesheet_ds)
ice6g_icesheet_analytics_ds.to_netcdf(f"ice6g_icesheet_analytics.nc")


In [3]:
glac1d_icesheet_ds = xr.open_dataset("paper1_data/icesheet_runs/glac1d/glac1d_icesheet.nc")
glac1d_icesheet_analytics_ds = lgm_icesheet_analytics(glac1d_icesheet_ds)
glac1d_icesheet_analytics_ds.to_netcdf(f"glac1d_icesheet_analytics.nc")


In [25]:
ice6g_icesheet_corners_ds = xr.open_dataset("ice6g_corners_icesheet.nc")
ice6g_icesheet_corners_analytics_ds = lgm_icesheet_analytics(ice6g_icesheet_corners_ds)
ice6g_icesheet_corners_analytics_ds.to_netcdf(f"ice6g_corners_icesheet_analytics.nc")


In [27]:
glac1d_icesheet_corners_ds = xr.open_dataset("glac1d_icesheet_corners.nc")
glac1d_icesheet_corners_analytics_ds = lgm_icesheet_analytics(glac1d_icesheet_corners_ds)
glac1d_icesheet_corners_analytics_ds.to_netcdf(f"glac1d_corners_icesheet_analytics.nc")


In [12]:
ice6g_ds = xr.open_dataset("paper1_data/ice_models/ice6g_laea_5km_eurasia.nc")
ice6g_analytics_ds = lgm_model_analytics(ice6g_ds, time=glac1d_icesheet_ds.time.values)
ice6g_analytics_ds.to_netcdf(f"ice6g_analytics.nc")


In [13]:
glac1d_ds = xr.open_dataset("paper1_data/ice_models/glac1d_laea_5km_eurasia.nc")
glac1d_analytics_ds = lgm_model_analytics(glac1d_ds, time=glac1d_icesheet_ds.time.values)
glac1d_analytics_ds.to_netcdf(f"glac1d_analytics.nc")


In [14]:
glac1d_analytics_ds

<xarray.Dataset>
Dimensions:                (region: 3, time: 4)
Coordinates:
  * time                   (time) int64 -22000 -20000 -18000 -16000
  * region                 (region) <U3 'bks' 'f' 'bi'
Data variables:
    ice_volume             (time) float64 7.033e+15 7.179e+15 ... 4.369e+15
    ice_volume_sle         (time) float64 17.35 17.71 13.85 10.78
    region_ice_volume      (region, time) float64 2.222e+15 ... 3.314e+13
    region_ice_volume_sle  (region, time) float64 5.482 6.149 ... 0.1167 0.08176

# Summary Metrics

In [ ]:
dA = (abs(batch_ds.x[1] - batch_ds.x[0]) * abs(batch_ds.y[1] - batch_ds.y[0])).values
shape = batch_ds.ice_thickness[0,0].shape
# barants-kara sea
mask_bks = np.zeros(shape)
mask_bks[588:, :] = 1

# fennoscandia
mask_f = np.zeros(shape)
mask_f[:588, 294:] = 1

# british-irish
mask_bi = np.zeros(shape)
mask_bi[:588, :294] = 1

In [ ]:
pgm_hist_match_ds = xr.open_dataset("/nfs/annie/cm15ogp/phd/paper1/analytics/history_match_pgm.nc")
ld_hist_match_ds = xr.open_dataset("/nfs/annie/cm15ogp/phd/paper1/analytics/hist_match_lgm_lhs.nc")

pgm_nroy = pgm_hist_match_ds.nroy.values
ld_nroy = ld_hist_match_ds.nroy.values

In [ ]:
icesheet_pgm_0_vol = xr.open_dataset("batchelor_topo_0_analytics.nc").ice_volume.values
icesheet_pgm_1_vol = xr.open_dataset("batchelor_topo_1_analytics.nc").ice_volume.values
icesheet_glac1d_vol = xr.open_dataset("glac1d_icesheet_analytics.nc").ice_volume.values
icesheet_ice6g_vol = xr.open_dataset("ice6g_icesheet_analytics.nc").ice_volume.values

icesheet_pgm_0_vol_reg = xr.open_dataset("batchelor_topo_0_analytics.nc").region_ice_volume.values
icesheet_pgm_1_vol_reg = xr.open_dataset("batchelor_topo_1_analytics.nc").region_ice_volume.values
icesheet_glac1d_vol_reg = xr.open_dataset("glac1d_icesheet_analytics.nc").region_ice_volume.values
icesheet_ice6g_vol_reg = xr.open_dataset("ice6g_icesheet_analytics.nc").region_ice_volume.values



In [ ]:
icesheet_glac1d_vol_mean, icesheet_glac1d_vol_std = icesheet_glac1d_vol.mean(axis=0), icesheet_glac1d_vol.std(axis=0)
icesheet_ice6g_vol_mean, icesheet_ice6g_vol_std = icesheet_ice6g_vol.mean(axis=0), icesheet_ice6g_vol.std(axis=0)
icesheet_glac1d_vol_nroy_mean, icesheet_glac1d_vol_nroy_std = icesheet_glac1d_vol[ld_nroy].mean(axis=0), icesheet_glac1d_vol[ld_nroy].std(axis=0)
icesheet_ice6g_vol_nroy_mean, icesheet_ice6g_vol_nroy_std = icesheet_ice6g_vol[ld_nroy].mean(axis=0), icesheet_ice6g_vol[ld_nroy].std(axis=0)

ld_vol_mean, ld_vol_std = np.vstack([icesheet_ice6g_vol_mean, icesheet_glac1d_vol_mean]).mean(axis=0), np.vstack([icesheet_ice6g_vol_mean, icesheet_glac1d_vol_mean]).std(axis=0)
ld_vol_nroy_mean, ld_vol_nroy_std = np.vstack([icesheet_ice6g_vol_nroy_mean, icesheet_glac1d_vol_nroy_mean]).mean(axis=0), np.vstack([icesheet_ice6g_vol_nroy_mean, icesheet_glac1d_vol_nroy_mean]).std(axis=0)

icesheet_glac1d_vol_reg_mean, icesheet_glac1d_vol_reg_std = icesheet_glac1d_vol_reg.mean(axis=1), icesheet_glac1d_vol_reg.std(axis=1)
icesheet_ice6g_vol_reg_mean, icesheet_ice6g_vol_reg_std = icesheet_ice6g_vol_reg.mean(axis=1), icesheet_ice6g_vol_reg.std(axis=1)
icesheet_glac1d_vol_reg_nroy_mean, icesheet_glac1d_vol_reg_nroy_std = icesheet_glac1d_vol_reg[:,ld_nroy].mean(axis=1), icesheet_glac1d_vol_reg[:,ld_nroy].std(axis=1)
icesheet_ice6g_vol_reg_nroy_mean, icesheet_ice6g_vol_reg_nroy_std = icesheet_ice6g_vol_reg[:,ld_nroy].mean(axis=1), icesheet_ice6g_vol_reg[:,ld_nroy].std(axis=1)

ld_vol_reg_mean, ld_vol_reg_std = np.asarray([icesheet_ice6g_vol_reg_mean, icesheet_glac1d_vol_reg_mean]).mean(axis=0), np.asarray([icesheet_ice6g_vol_reg_mean, icesheet_glac1d_vol_reg_mean]).std(axis=0)
ld_vol_reg_nroy_mean, ld_vol_reg_nroy_std = np.asarray([icesheet_ice6g_vol_reg_nroy_mean, icesheet_glac1d_vol_reg_nroy_mean]).mean(axis=0), np.asarray([icesheet_ice6g_vol_reg_nroy_mean, icesheet_glac1d_vol_reg_nroy_mean]).std(axis=0)


In [ ]:
icesheet_pgm_0_vol_mean, icesheet_pgm_0_vol_std = icesheet_pgm_0_vol.mean(), icesheet_pgm_0_vol.std()
icesheet_pgm_1_vol_mean, icesheet_pgm_1_vol_std = icesheet_pgm_1_vol.mean(), icesheet_pgm_1_vol.std()

icesheet_pgm_0_vol_mean, icesheet_pgm_0_vol_std = icesheet_pgm_0_vol.mean(axis=0), icesheet_pgm_0_vol.std(axis=0)
icesheet_pgm_1_vol_mean, icesheet_pgm_1_vol_std = icesheet_pgm_1_vol.mean(axis=0), icesheet_pgm_1_vol.std(axis=0)
icesheet_pgm_0_vol_nroy_mean, icesheet_pgm_0_vol_nroy_std = icesheet_pgm_0_vol[pgm_nroy].mean(axis=0), icesheet_pgm_0_vol[pgm_nroy].std(axis=0)
icesheet_pgm_1_vol_nroy_mean, icesheet_pgm_1_vol_nroy_std = icesheet_pgm_1_vol[pgm_nroy].mean(axis=0), icesheet_pgm_1_vol[pgm_nroy].std(axis=0)

icesheet_pgm_0_vol_reg_mean, icesheet_pgm_0_vol_reg_std = icesheet_pgm_0_vol_reg.mean(axis=1), icesheet_pgm_0_vol_reg.std(axis=1)
icesheet_pgm_1_vol_reg_mean, icesheet_pgm_1_vol_reg_std = icesheet_pgm_1_vol_reg.mean(axis=1), icesheet_pgm_1_vol_reg.std(axis=1)
icesheet_pgm_0_vol_reg_nroy_mean, icesheet_pgm_0_vol_reg_nroy_std = icesheet_pgm_0_vol_reg[:,pgm_nroy].mean(axis=1), icesheet_pgm_0_vol_reg[:,pgm_nroy].std(axis=1)
icesheet_pgm_1_vol_reg_nroy_mean, icesheet_pgm_1_vol_reg_nroy_std = icesheet_pgm_1_vol_reg[:,pgm_nroy].mean(axis=1), icesheet_pgm_1_vol_reg[:,pgm_nroy].std(axis=1)



In [ ]:

data = {}
ld_times = ['22ka', '20ka', '18ka', '16ka']
for idx, ld_time in enumerate(ld_times):
    data[f'icesheet_glac1d_{ld_time}'] = [icesheet_glac1d_vol_mean[idx], icesheet_glac1d_vol_std[idx], *icesheet_glac1d_vol_reg_mean[:,idx], *icesheet_glac1d_vol_reg_std[:,idx]]
    data[f'icesheet_ice6g_{ld_time}'] = [icesheet_ice6g_vol_mean[idx], icesheet_ice6g_vol_std[idx], *icesheet_ice6g_vol_reg_mean[:,idx], *icesheet_ice6g_vol_reg_std[:,idx]]
    data[f'icesheet_ld_{ld_time}'] = [ld_vol_mean[idx], ld_vol_std[idx], *ld_vol_reg_mean[:,idx], *ld_vol_reg_std[:,idx]]
    data[f'icesheet_glac1d_nroy_{ld_time}'] = [icesheet_glac1d_vol_nroy_mean[idx], icesheet_glac1d_vol_nroy_std[idx], *icesheet_glac1d_vol_reg_nroy_mean[:,idx], *icesheet_glac1d_vol_reg_nroy_std[:,idx]]
    data[f'icesheet_ice6g_nroy_{ld_time}'] = [icesheet_ice6g_vol_nroy_mean[idx], icesheet_ice6g_vol_nroy_std[idx], *icesheet_ice6g_vol_reg_nroy_mean[:,idx], *icesheet_ice6g_vol_reg_nroy_std[:,idx]]
    data[f'icesheet_ld_nroy_{ld_time}'] = [ld_vol_nroy_mean[idx], ld_vol_nroy_std[idx], *ld_vol_reg_nroy_mean[:,idx], *ld_vol_reg_nroy_std[:,idx]]

data[f'icesheet_pgm_topo_0'] = [icesheet_pgm_0_vol_mean[0], icesheet_pgm_0_vol_std[0], *icesheet_pgm_0_vol_reg_mean[:,0], *icesheet_pgm_0_vol_reg_std[:,0]]
data[f'icesheet_pgm_topo_1'] = [icesheet_pgm_1_vol_mean[0], icesheet_pgm_1_vol_std[0], *icesheet_pgm_1_vol_reg_mean[:,0], *icesheet_pgm_1_vol_reg_std[:,0]]
data[f'icesheet_pgm_topo_0_nroy'] = [icesheet_pgm_0_vol_nroy_mean[0], icesheet_pgm_0_vol_nroy_std[0], *icesheet_pgm_0_vol_reg_nroy_mean[:,0], *icesheet_pgm_0_vol_reg_nroy_std[:,0]]
data[f'icesheet_pgm_topo_1_nroy'] = [icesheet_pgm_1_vol_nroy_mean[0], icesheet_pgm_1_vol_nroy_std[0], *icesheet_pgm_1_vol_reg_nroy_mean[:,0], *icesheet_pgm_1_vol_reg_nroy_std[:,0]]

columns = ['total_mean', 'total_std'] + [f'{region}_mean' for region in ['bks', 'f', 'bi']] + [f'{region}_std' for region in ['bks', 'f', 'bi']]
df = pd.DataFrame.from_dict(data, orient='index', columns=columns).sort_index()
df = df[['total_mean','total_std','bks_mean','bks_std','f_mean','f_std','bi_mean','bi_std']]

In [ ]:
df_sle = df.applymap(volume_to_sle).round(2)

In [ ]:
df.to_csv("volume_summary_metrics.csv")
df_sle.to_csv("sle_volume_summary_metrics.csv")

# Volume Laurentide Calc

In [18]:
import xarray as xr
import numpy as np
from icepy import calc_grid_area_geo, volume_to_sle, region_mask

In [19]:
def calc_vol(er_pgm):
    waelbrock_diff = waelbrock_pgm - (na_lgm + an_lgm + gr_lgm + er_pgm)
    perc_diff = 100 - (100*((na_lgm - waelbrock_diff)/na_lgm))
    return perc_diff

In [20]:
ds_6g = xr.open_dataset("../../../../data/ice_models/ice6g/ice6g_latlon_1deg.nc").ice_thickness
ds_1d = xr.open_dataset("../../../../data/ice_models/glac1d/glac1d_latlon_latp25lonp5.nc").ice_thickness
a_6g = calc_grid_area_geo(ds_6g.lon.values, ds_6g.lat.values)
a_1d = calc_grid_area_geo(ds_1d.lon.values, ds_1d.lat.values)

In [21]:
present = (volume_to_sle(np.sum(ds_6g.sel(time=0).values * a_6g)) + volume_to_sle(np.sum(ds_1d.sel(time=0).values * a_1d)))/2
waelbrock_pgm = 128.94 + present

gr_lgm_6g = volume_to_sle(np.sum(a_6g * ds_6g.sel(time=-22000).values * region_mask(ds_6g, "Greenland")))
gr_lgm_1d = volume_to_sle(np.sum(a_1d * ds_1d.sel(time=-22000).values * region_mask(ds_1d, "Greenland")))
gr_lgm = (gr_lgm_6g + gr_lgm_1d)/2

an_lgm_6g = volume_to_sle(np.sum(a_6g * ds_6g.sel(time=-22000).values * region_mask(ds_6g, "Antarctica")))
an_lgm_1d = volume_to_sle(np.sum(a_1d * ds_1d.sel(time=-22000).values * region_mask(ds_1d, "Antarctica")))
an_lgm = (an_lgm_6g + an_lgm_1d)/2

na_lgm_6g = volume_to_sle(np.sum(a_6g * ds_6g.sel(time=-22000).values * region_mask(ds_6g, "North America")))
na_lgm_1d = volume_to_sle(np.sum(a_1d * ds_1d.sel(time=-22000).values * region_mask(ds_1d, "North America")))
na_lgm = (na_lgm_6g + na_lgm_1d)/2


In [24]:
calc_vol(48)

-9.938913158064565